In [1]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from os import listdir
import cv2
import math
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from collections import Counter
from tqdm import tqdm

### The data is stored inside the 'team' directory

In [2]:
path = '../../team/courses/MSiA400/GrandTeton'

In [3]:
path_real = path + '/Photos-20191020T020916Z-001/Photos/'
path_generated_go = path + '/GO_noGO Data Set_Images/TestGo/'
path_generated_nogo = path + '/GO_noGO Data Set_Images/TestNoGo/'

# Set path for training images
filepaths_go = [f for f in listdir(path_generated_go) if f.endswith('.png')]
filepaths_nogo = [f for f in listdir(path_generated_nogo) if f.endswith('.png')]

In [4]:
# Read all GO and NoGO images into a list
list_img = []
for i in tqdm(filepaths_go):
    list_img.append(cv2.imread(path_generated_go + i))

100%|██████████| 3370/3370 [00:28<00:00, 117.47it/s]


In [5]:
for i in tqdm(filepaths_nogo):
    list_img.append(cv2.imread(path_generated_nogo + i))

100%|██████████| 5001/5001 [00:46<00:00, 107.63it/s]


In [6]:
n = len(list_img)

In [7]:
n

8371

### Feature Extraction class

In [8]:
class Feature_Extractor:
    # class defines feature extraction methods for extracting features out of an image
    def __init__(self, threshold = 200, line_margin = 15, roof_margin = 10, corner_margin = 5):
        self.threshold = threshold
        self.line_margin = line_margin
        self.roof_margin = roof_margin
        self.corner_margin = corner_margin

    def img_width(self, img):
        return img.shape[1]
    
    def img_height(self, img):
        return img.shape[0]

    def count_level(self, img):
        # Function that counts the number of floors
        # Convert img to grayscale
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Get shape
        shape = gray.shape

        # Get width of the image
        width = shape[1]

        # Detect edges
        edges = cv2.Canny(gray, 80, 120)

        #HoughLinesP returns an array of (rho, theta) values. 
        #rho is measured in pixels and theta is measured in radians

        # Detect lines representing the floors which are longer than 80% of the width of the building
        lines = cv2.HoughLinesP(edges, rho = 1, theta = math.pi/2, minLineLength = 0.8*width, threshold = 1, maxLineGap = 3)
        
        flags = [0]
        if (type(lines) != type(None)):
            lines.tolist()

            # Delete repeated lines and line detected from the roof (we only need floor lines to count the number of floors)

            flags = [0]*len(lines)  # flags will mark the redundant lines as 1, lines we need as 0
            for i in range(len(lines)):
                for j in range(len(lines)):
                    if j < i and (abs(lines[i][0][1]-lines[j][0][1]) < self.line_margin):  # detect lines very close to each other 
                        flags[j] = 1
                if abs(lines[i][0][1]-0) < self.roof_margin:  # roof lines: y Coordinate -> 0
                    flags[i] = 1
        counter = 0
        for i in range(len(flags)):
            if flags[i] == 0:
                counter += 1                
        return counter

    def count_openings(self, img):
        # Function that counts the number of openings in a house
        # most of the openings are white rectangles which denote windows and gates
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        height = gray.shape[0]
        width = gray.shape[1]

        # threshold optimized (via hit and trial) for the generated images
        ret,thresh = cv2.threshold(gray, self.threshold, 255, 0)
        contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        quadrilaterals = []
        for i in range(len(contours)):
            polygon = cv2.approxPolyDP(contours[i],0.01*cv2.arcLength(contours[i],True),True)
            if len(polygon) == 4:
                quadrilaterals.append(polygon) 
        # Detect and delete quadrilaterals outside the house which should not be counted as openings
        redflag = [0]*len(quadrilaterals)

        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            for j in range(4):
                if abs(q[j][0][0] - width) < self.corner_margin or abs(q[j][0][0]) < self.corner_margin:
                    redflag[i] = 1

        return (len(quadrilaterals) - np.sum(redflag))

    def fraction_width(self, img):
        # Function that calculates proportion of sum of all windows' widths (without overlap), on all floors 
        # to the overall wigth of building
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        height = gray.shape[0]
        width = gray.shape[1]

        # threshold optimized (via hit and trial) for the generated images
        ret, thresh = cv2.threshold(gray, self.threshold, 255, 0)

        # Contours is a tree of lists of points which describe each contour
        contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Create a list storing quadrilaterals that represent openings
        quadrilaterals = []
        for i in range(len(contours)):

            # Contour approximation will mark four vertices of a quadrilateral
            polygon = cv2.approxPolyDP(contours[i],0.01*cv2.arcLength(contours[i],True),True)
            # filtering only those openings that are quadrilaterals
            if len(polygon) == 4:
                quadrilaterals.append(polygon) 

        redflag = [0]*len(quadrilaterals)

        # Detect and delete quadrilaterals outside the house which should not be counted as openings
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            for j in range(4):
                if abs(q[j][0][0] - width) < self.corner_margin or abs(q[j][0][0]) < self.corner_margin:
                    redflag[i] = 1

        # Get a blank canvas for drawing width of a side of each quadrilateral
        detection_series = np.zeros(width, dtype = 'uint8')

        # The width of a side should be the larger x cordinate of the right vertics 
        # minus the x cordinate of the left vertics
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            if redflag[i]!=1:
                x_min = np.min(q[:,0,0])
                x_max = np.max(q[:,0,0])
                detection_series[x_min:x_max] = np.ones(x_max-x_min, dtype = 'uint8')

        # Return fraction of sum of all windows' widths (without overlap), on all floors to the overall width of building
        return np.sum(detection_series)/width
    
    def avg_fraction_width(self, img):
        # Function that calculates proportion of sum of all windows' widths (divided by the number of floors) 
        # to the overall length of building

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        height = gray.shape[0]
        width = gray.shape[1]

        # threshold optimized (via hit and trial) for the generated images
        ret, thresh = cv2.threshold(gray, self.threshold, 255, 0)

        # Contours is a tree of lists of points which describe each contour
        contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Create a list storing quadrilaterals that represent openings
        quadrilaterals = []
        for i in range(len(contours)):

            # Contour approximation will mark four vertices of a quadrilateral
            polygon = cv2.approxPolyDP(contours[i],0.01*cv2.arcLength(contours[i],True),True)
            # filtering only those openings that are quadrilaterals
            if len(polygon) == 4:
                quadrilaterals.append(polygon) 

        redflag = [0]*len(quadrilaterals)

        # Detect and delete quadrilaterals outside the house which should not be counted as openings
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            for j in range(4):
                if abs(q[j][0][0] - width) < self.corner_margin or abs(q[j][0][0]) < self.corner_margin:
                    redflag[i] = 1

        # set aggregate width = 0 before the loop that is going to account for the width of each opening
        aggregate_width = 0;

        # The width of a side should be the larger x cordinate of the right vertics 
        # minus the x cordinate of the left vertics
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            if redflag[i]!=1:
                x_min = np.min(q[:,0,0])
                x_max = np.max(q[:,0,0])
                aggregate_width = aggregate_width + (x_max-x_min)

        # now in order to calculate the average, we need the number of floors
        num_levels = self.count_level(img)

        # Return the ratio of: average of sum of all windows' widths (over all floors) to the total width of the building
        return aggregate_width/(num_levels*width)
    
    def fraction_height(self, img):
        # Function that calculates proportion of sum of all windows' heights (without overlap), on all floors
        # to the overall length of building
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        height = gray.shape[0]
        width = gray.shape[1]

        # threshold optimized (via hit and trial) for the generated images
        ret, thresh = cv2.threshold(gray, self.threshold, 255, 0)

        # Contours is a tree of lists of points which describe each contour
        contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Create a list storing quadrilaterals that represent openings
        quadrilaterals = []
        for i in range(len(contours)):

            # Contour approximation will mark four vertices of a quadrilateral
            polygon = cv2.approxPolyDP(contours[i],0.01*cv2.arcLength(contours[i],True),True)
            # filtering only those openings that are quadrilaterals
            if len(polygon) == 4:
                quadrilaterals.append(polygon) 

        redflag = [0]*len(quadrilaterals)

        # Detect and delete quadrilaterals outside the house which should not be counted as openings
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            for j in range(4):
                if abs(q[j][0][0] - gray.shape[1]) < self.corner_margin or abs(q[j][0][0]) < self.corner_margin:
                    redflag[i] = 1

        # Get a blank canvas for drawing width of a side of each quadrilateral
        detection_series = np.zeros(height, dtype = 'uint8')

        # The height of a side should be the larger y cordinate of the top vertics 
        # minus the y cordinate of the bottom vertics
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            if redflag[i]!=1:
                y_min = np.min(q[:,0,1])
                y_max = np.max(q[:,0,1])
                detection_series[y_min:y_max] = np.ones(y_max-y_min, dtype = 'uint8')

        # Return fraction of sum of all windows' heights (without overlap), on all floors to the overall length of building
        return np.sum(detection_series)/height
    
    def aggregate_fraction_height(self, img):
        # Function that calculates proportion of sum of all windows' heights (divided by the number of floors) 
        # to the overall length of building
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        height = gray.shape[0]
        width = gray.shape[1]

        # threshold optimized (via hit and trial) for the generated images
        ret, thresh = cv2.threshold(gray, self.threshold, 255, 0)

        # Contours is a tree of lists of points which describe each contour
        contours, h = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Create a list storing quadrilaterals that represent openings
        quadrilaterals = []
        for i in range(len(contours)):

            # Contour approximation will mark four vertices of a quadrilateral
            polygon = cv2.approxPolyDP(contours[i],0.01*cv2.arcLength(contours[i],True),True)
            # filtering only those openings that are quadrilaterals
            if len(polygon) == 4:
                quadrilaterals.append(polygon) 

        redflag = [0]*len(quadrilaterals)

        # Detect and delete quadrilaterals outside the house which should not be counted as openings
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            for j in range(4):
                if abs(q[j][0][0] - gray.shape[1]) < self.corner_margin or abs(q[j][0][0]) < self.corner_margin:
                    redflag[i] = 1

        # set aggregate height = 0 before the loop that is going to account for the height of each opening
        aggregate_height = 0

        # The height of a side should be the larger y cordinate of the top vertics 
        # minus the y cordinate of the bottom vertics
        for i in range(len(quadrilaterals)):
            q = quadrilaterals[i]
            if redflag[i]!=1:
                y_min = np.min(q[:,0,1])
                y_max = np.max(q[:,0,1])
                aggregate_height = aggregate_height + (y_max-y_min)

        # To be careful:
        # Width of each floor is same and equal to the width of the house
        # However, height of each floor = height of the house / 3
        # So we actually don't need the number of floors to calculate the average

        # there is no notion of vertical floors, so this ratio is going to exceed one
        # Return the ratio of: sum of all windows' height to the total height of the building
        return aggregate_height/height


### Processing feature vectors for all the images

In [9]:
feature_extractor = Feature_Extractor()

In [10]:
# Create a list called 'levels' to store number of floors for each building
levels = []
for i in tqdm(range(n)):
    n_level = feature_extractor.count_level(list_img[i])
    levels.append(n_level)

100%|██████████| 8371/8371 [00:09<00:00, 871.25it/s]


In [11]:
Counter(levels)

Counter({1: 2037, 3: 3763, 2: 2536, 4: 35})

In [12]:
# Create a list called 'openings' to store number of openings
openings = []
for i in tqdm(range(n)):
    openings.append(feature_extractor.count_openings(list_img[i]))

100%|██████████| 8371/8371 [00:05<00:00, 1403.04it/s]


In [13]:
Counter(openings)

Counter({1: 628,
         2: 1475,
         4: 1056,
         6: 1791,
         3: 1654,
         5: 389,
         7: 276,
         9: 881,
         8: 220,
         0.0: 1})

In [14]:
# Create a list called 'fraction_widths' to store proportion of sum of all windows' widths (without overlap), (on all floors) 
# to the overall width of building
fraction_widths = []
for i in tqdm(range(len(list_img))):
    fraction_widths.append(feature_extractor.fraction_width(list_img[i]))

100%|██████████| 8371/8371 [00:06<00:00, 1232.36it/s]


In [15]:
# Create a list called 'avg_fraction_widths' to store proportion of average of all windows' widths (over all floors) 
# to the overall width of building
avg_fraction_widths = []
for i in tqdm(range(len(list_img))):
    avg_fraction_widths.append(feature_extractor.avg_fraction_width(list_img[i]))

100%|██████████| 8371/8371 [00:16<00:00, 518.21it/s]


In [16]:
# Create a list called 'fraction_heights' to store proportion of sum of all windows' heights (without overlap), on all floors 
# to the overall height of building
fraction_heights = []
for i in tqdm(range(len(list_img))):
    fraction_heights.append(feature_extractor.fraction_height(list_img[i]))

100%|██████████| 8371/8371 [00:06<00:00, 1220.81it/s]


In [17]:
# Create a list called 'aggregate_fraction_heights' to store proportion of sum of all windows' heights (on all floors) 
# to the overall height of building
aggregate_fraction_heights = []
for i in tqdm(range(len(list_img))):
    aggregate_fraction_heights.append(feature_extractor.aggregate_fraction_height(list_img[i]))

100%|██████████| 8371/8371 [00:06<00:00, 1339.13it/s]


In [18]:
# Create a list called 'img_widths' to store the pixel widths of all images
img_widths = []
for i in tqdm(range(len(list_img))):
    img_widths.append(feature_extractor.img_width(list_img[i]))

100%|██████████| 8371/8371 [00:00<00:00, 555704.45it/s]


In [19]:
# Create a list called 'img_heights' to store the pixel widths of all images
img_heights = []
for i in tqdm(range(len(list_img))):
    img_heights.append(feature_extractor.img_height(list_img[i]))

100%|██████████| 8371/8371 [00:00<00:00, 570105.52it/s]


In [20]:
# Extract image index (four digit number)
files_go_idx = []
for file in filepaths_go:
    files_go_idx.append(int(file.split("Img")[1].split(".")[0]))
files_go_idx[:10]

[1007, 1008, 1011, 1016, 1026, 103, 1031, 1039, 1047, 105]

In [21]:
files_nogo_idx = []
for file in filepaths_nogo:
    if "Img" in file:
        files_nogo_idx.append(int(file.split("Img")[1].split(".")[0]))
files_nogo_idx[:10]

[1, 10, 100, 1000, 1001, 1002, 1003, 1004, 1005, 1006]

In [22]:
files = files_go_idx + files_nogo_idx

In [23]:
# Create a a dataframe with all features and image index as columns
dic = {"filename":files, "levels":levels, "openings":openings, "fraction_widths":fraction_widths, 
       "avg_fraction_widths":avg_fraction_widths, "fraction_heights":fraction_heights, 
       "aggregate_fraction_heights":aggregate_fraction_heights, "img_widths":img_widths, "img_heights":img_heights}
df = pd.DataFrame(dic)

In [24]:
df.head()

,filename,levels,openings,fraction_widths,avg_fraction_widths,fraction_heights,aggregate_fraction_heights,img_widths,img_heights
0,1007,1,1.0,0.096045,0.096045,0.407767,0.407767,531,206
1,1008,1,1.0,0.105461,0.105461,0.432990,0.432990,531,194
2,1011,1,2.0,0.167608,0.167608,0.461165,0.665049,531,206
3,1016,3,4.0,0.269492,0.089831,0.336927,0.412399,590,371
4,1026,2,2.0,0.133710,0.097928,0.410494,0.410494,531,324


In [25]:
# Add GO/NoGo column to label each image
df['Go/NoGo']=df['filename'].apply(lambda x: 1 if x in files_go_idx else 0)

In [26]:
df.head()

,filename,levels,openings,fraction_widths,avg_fraction_widths,fraction_heights,aggregate_fraction_heights,img_widths,img_heights,Go/NoGo
0,1007,1,1.0,0.096045,0.096045,0.407767,0.407767,531,206,1
1,1008,1,1.0,0.105461,0.105461,0.432990,0.432990,531,194,1
2,1011,1,2.0,0.167608,0.167608,0.461165,0.665049,531,206,1
3,1016,3,4.0,0.269492,0.089831,0.336927,0.412399,590,371,1
4,1026,2,2.0,0.133710,0.097928,0.410494,0.410494,531,324,1


### Fitting in logistic model and evaluate the performance(accuracy)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc [:, 1:9] , df['Go/NoGo'], test_size=0.3, random_state=0)

In [29]:
## normalizing the feature vectors
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train))
X_test_scaled = pd.DataFrame(scaler.transform(X_test))

In [30]:
## hyperparameter tuning
logreg = LogisticRegression(solver = 'liblinear', class_weight = 'balanced')
# balanced weights for class imbalance
penalty = ['l1', 'l2']
C =np.logspace(-2, 4, 10)
hyperparameters = dict(C=C, penalty=penalty)

In [31]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logreg, hyperparameters, cv=5, verbose=0)

In [32]:
best_model = clf.fit(X_train_scaled, y_train)

In [33]:
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])

Best Penalty: l1
Best C: 0.046415888336127774


In [34]:
y_pred = best_model.predict(X_test_scaled)

In [35]:
print('Accuracy of logistic regression classifier on test set: {:.2f}%'.format(100*best_model.score(X_test_scaled, y_test)))

Accuracy of logistic regression classifier on test set: 79.66%


In [36]:
## Confustion Matrix
confusion_matrix(y_test, y_pred)

array([[1219,  296],
       [ 215,  782]])